#Обработка данных

###Импорт библиотек и данных

> Добавить блок с цитатой



In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
history=pd.read_csv('https://drive.google.com/uc?export=download&id=1SmFlAHKnaqPy-lp8H3ZPegqQpz6C6gUq', sep='\t')
users=pd.read_csv('https://drive.google.com/uc?export=download&id=16UVDTOX6Gli4amZnDsRrow11ywI-WHl2', sep='\t')
validate_answers=pd.read_csv('https://drive.google.com/uc?export=download&id=1Nz-z3heM8zhUYahcpcsQ7wXxYUo-Ud2B', sep='\t')
validate=pd.read_csv('https://drive.google.com/uc?export=download&id=121sJ7OJ-ZXKAElaXiNHNsuZ49Qmbd2ki', sep='\t')

###Объединение датасетов + новые признаки

In [30]:
history['hour_mod_24'] = history['hour'] % 24

In [31]:
def create_users_parametrs(user_ids):
    user_ = users[users['user_id'].isin(user_ids)]

    if user_.empty:
        return pd.Series({
            'average_age': 0,
            'percent_6_15': 0,
            'percent_16_25': 0,
            'percent_26_60': 0,
            'percent_61_75': 0,
            'percent_men': 0,
            'unique_cities': 0,
            'percent_cities_3_7_19_25': 0
        })

    average_age = user_['age'].mean()
    total_users = len(user_)
    percent_6_15 = (user_[(user_['age'] >= 6) & (user_['age'] <= 15)].shape[0] / total_users) * 100
    percent_16_25 = (user_[(user_['age'] >= 16) & (user_['age'] <= 25)].shape[0] / total_users) * 100
    percent_26_60 = (user_[(user_['age'] >= 26) & (user_['age'] <= 60)].shape[0] / total_users) * 100
    percent_61_75 = (user_[(user_['age'] >= 61) & (user_['age'] <= 75)].shape[0] / total_users) * 100

    percent_men = (user_[user_['sex'] == 1].shape[0] / total_users) * 100

    unique_cities = user_['city_id'].nunique()
    cities_of_interest = [3, 7, 19, 25]
    percent_big_cities = (user_[user_['city_id'].isin(cities_of_interest)].shape[0] / total_users) * 100

    return pd.Series({
        'average_age': average_age,
        'percent_6_15': percent_6_15,
        'percent_16_25': percent_16_25,
        'percent_26_60': percent_26_60,
        'percent_61_75': percent_61_75,
        'percent_men': percent_men,
        'unique_cities': unique_cities,
        'percent_cities_3_7_19_25': percent_big_cities
    })

In [32]:
result = validate.merge(validate_answers, left_index=True, right_index=True)
result['user_ids'] = result['user_ids'].apply(lambda x: eval(x))
params = result['user_ids'].apply(create_users_parametrs)
result = pd.concat([result, params], axis=1)

In [33]:
def create_publishers_parametrs(publishers):
    if isinstance(publishers, str):
        publishers = eval(publishers)
    elif isinstance(publishers, int):
        publishers = [publishers]

    filtered_history = history[history['publisher'].isin(publishers)]

    if filtered_history.empty:
        return pd.Series({
            'average_cpm': 0,
            'min_cpm': 0,
            'max_cpm': 0,
            'most_active_hour': None,
            'second_active_hour': None,
            'third_active_hour': None,
            'fourth_active_hour': None,
            'fifth_active_hour': None
        })

    average_cpm = filtered_history['cpm'].mean()
    min_cpm = filtered_history['cpm'].min()
    max_cpm = filtered_history['cpm'].max()

    active_hours = filtered_history['hour_mod_24'].value_counts().nlargest(5)

    most_active_hour = active_hours.index[0] if len(active_hours) > 0 else None
    second_active_hour = active_hours.index[1] if len(active_hours) > 1 else None
    third_active_hour = active_hours.index[2] if len(active_hours) > 2 else None
    fourth_active_hour = active_hours.index[3] if len(active_hours) > 3 else None
    fifth_active_hour = active_hours.index[4] if len(active_hours) > 4 else None

    return pd.Series({
        'average_cpm': average_cpm,
        'min_cpm': min_cpm,
        'max_cpm': max_cpm,
        'most_active_hour': most_active_hour,
        'second_active_hour': second_active_hour,
        'third_active_hour': third_active_hour,
        'fourth_active_hour': fourth_active_hour,
        'fifth_active_hour': fifth_active_hour
    })

In [34]:
result['publishers'] = result['publishers'].apply(lambda x: eval(x))
cpm_metrics = result['publishers'].apply(create_publishers_parametrs)
result = pd.concat([result, cpm_metrics], axis=1)

In [54]:
result['publishers'] = result['publishers'].astype(str)
result = result.assign(n_publishers=result.publishers.apply(lambda s: len(s.split(','))),
                 duration=result.hour_end - result.hour_start)

In [53]:
result

,cpm,hour_start,hour_end,publishers,audience_size,user_ids,at_least_one,at_least_two,at_least_three,average_age,...,average_cpm,min_cpm,max_cpm,most_active_hour,second_active_hour,third_active_hour,fourth_active_hour,fifth_active_hour,duration,n_publishers
0,220.0,1058,1153,"(7, 17)",1906,"(12, 44, 46, 50, 58, 71, 93, 122, 134, 143, 17...",0.0430,0.0152,0.0073,28.721406,...,243.344230,30.0,33147.27,13.0,15.0,14.0,16.0,11.0,95,2
1,312.0,1295,1301,"(3, 18)",1380,"(29, 81, 98, 102, 165, 167, 195, 205, 218, 231...",0.0130,0.0000,0.0000,28.611594,...,195.653509,30.0,209053.98,21.0,20.0,19.0,22.0,18.0,6,2
2,70.0,1229,1249,"(1, 2, 3, 9, 15, 21)",888,"(12, 23, 25, 29, 45, 85, 92, 124, 156, 190, 27...",0.0878,0.0135,0.0000,29.055180,...,184.705490,30.0,209053.98,21.0,20.0,19.0,18.0,22.0,20,6
3,240.0,1295,1377,"(1, 14)",440,"(44, 122, 187, 209, 242, 255, 312, 345, 382, 4...",0.2295,0.1295,0.0727,27.763636,...,177.597601,30.0,64282.98,20.0,21.0,19.0,18.0,17.0,82,2
4,262.0,752,990,"(1, 3, 7, 8)",1476,"(15, 24, 30, 43, 50, 53, 96, 105, 159, 168, 18...",0.3963,0.2785,0.2270,44.760840,...,184.183367,30.0,209053.98,21.0,20.0,19.0,18.0,17.0,238,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,127.0,1095,1154,"(2, 7, 12, 15, 16)",368,"(38, 50, 143, 237, 456, 649, 653, 698, 881, 89...",0.0679,0.0408,0.0190,28.891304,...,210.098451,30.0,55909.62,21.0,20.0,22.0,19.0,18.0,59,5
1004,90.0,1052,1056,"(2, 21)",484,"(1, 4, 61, 212, 230, 348, 356, 372, 429, 431, ...",0.0165,0.0000,0.0000,28.954545,...,203.129447,30.0,55909.62,21.0,20.0,22.0,19.0,18.0,4,2
1005,122.0,1058,1063,"(2, 5)",704,"(1, 23, 29, 104, 136, 176, 201, 206, 250, 269,...",0.0071,0.0000,0.0000,28.169034,...,202.636239,30.0,55909.62,21.0,20.0,22.0,19.0,18.0,5,2
1006,138.0,932,1169,"(1, 2, 7)",1210,"(67, 73, 80, 146, 182, 202, 212, 254, 307, 377...",0.3669,0.2628,0.2066,88.629752,...,188.622220,30.0,64282.98,21.0,20.0,19.0,18.0,22.0,237,3


In [85]:
from typing import Tuple
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
import warnings

In [73]:
def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log(
        (responses_column + epsilon)
        / (answers_column + epsilon)
    )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = np.array(
        [
            get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(
        decimals=2
    )

def custom_scorer(y_true, y_pred):
    responses = pd.DataFrame(y_pred, columns=['at_least_one', 'at_least_two', 'at_least_three'])
    answers = y_true.reset_index(drop=True)
    return get_smoothed_mean_log_accuracy_ratio(answers, responses)

custom_scorer_func = make_scorer(custom_scorer, greater_is_better=False)

In [74]:
def make_xy(data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    X = result.drop(columns=['at_least_one', 'at_least_two', 'at_least_three', 'user_ids', 'publishers'])
    y = result[['at_least_one', 'at_least_two', 'at_least_three']]
    return X, y

In [112]:
X, y = make_xy(result)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

responses_train = pd.DataFrame(y_pred_train, columns=['at_least_one', 'at_least_two', 'at_least_three'])
answers_train = y_train.reset_index(drop=True)
responses_test = pd.DataFrame(y_pred_test, columns=['at_least_one', 'at_least_two', 'at_least_three'])
answers_test = y_test.reset_index(drop=True)

percentage_error_train = get_smoothed_mean_log_accuracy_ratio(answers_train, responses_train)
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(answers_test, responses_test)
print(f'Percentage Error on Train: {percentage_error_train}%')
print(f'Percentage Error on Test: {percentage_error_test}%')

Percentage Error on Train: 189.75%
Percentage Error on Test: 213.18%


In [113]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import make_scorer

In [87]:
warnings.filterwarnings("ignore", category=RuntimeWarning)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    'Ridge Regression': (Ridge(), {'alpha': [0.1, 0.2, 0.5, 0.7, 1.0, 2, 4, 5, 10, 15]}),
    'Lasso Regression': (Lasso(), {'alpha': [0.1, 0.2, 0.5, 0.7, 1.0, 2, 4, 5, 10, 15]}),
    'Elastic Net': (ElasticNet(), {'alpha': [0.1, 0.2, 0.5, 0.7, 1.0, 2, 4, 5, 10, 15], 'l1_ratio': [0.1, 0.2, 0.3, 0.5, 0.7, 0.85, 1.0]})
}

error_res = {}

for model_name, (model, params) in models.items():
    grid_search = GridSearchCV(model, params, scoring=custom_scorer_func, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    best_model = grid_search.best_estimator_

    train_predictions = best_model.predict(X_train_scaled)
    train_responses = pd.DataFrame(train_predictions, columns=['at_least_one', 'at_least_two', 'at_least_three'])
    train_answers = y_train.reset_index(drop=True)

    test_predictions = best_model.predict(X_test_scaled)
    test_responses = pd.DataFrame(test_predictions, columns=['at_least_one', 'at_least_two', 'at_least_three'])
    test_answers = y_test.reset_index(drop=True)

    train_error = get_smoothed_mean_log_accuracy_ratio(train_answers, train_responses)
    test_error = get_smoothed_mean_log_accuracy_ratio(test_answers, test_responses)

    error_res[model_name] = {
        'best_params': grid_search.best_params_,
        'train_error': train_error,
        'test_error': test_error
    }

for model_name, errors in error_res.items():
    print(f"{model_name}: Best Params = {errors['best_params']}, Train Error = {errors['train_error']}%, Test Error = {errors['test_error']}%")

Ridge Regression: Best Params = {'alpha': 0.1}, Train Error = 189.57%, Test Error = 212.28%
Lasso Regression: Best Params = {'alpha': 0.1}, Train Error = 361.4%, Test Error = 367.02%
Elastic Net: Best Params = {'alpha': 0.1, 'l1_ratio': 0.1}, Train Error = 202.11%, Test Error = 214.1%


In [88]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [94]:
models = {
    'Decision Tree': (DecisionTreeRegressor(), {'max_depth': [None, 1, 2, 5, 7, 10, 12, 15, 20]}),
    'Random Forest': (RandomForestRegressor(), {'n_estimators': [5, 10, 25, 50, 75, 100, 150], 'max_depth': [None,1, 2, 5, 7, 10, 12, 15, 20]})
}

error_res = {}

for model_name, (model, params) in models.items():
    grid_search = GridSearchCV(model, params, scoring=custom_scorer_func, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    best_model = grid_search.best_estimator_

    train_predictions = best_model.predict(X_train_scaled)
    train_responses = pd.DataFrame(train_predictions, columns=['at_least_one', 'at_least_two', 'at_least_three'])
    train_answers = y_train.reset_index(drop=True)

    test_predictions = best_model.predict(X_test_scaled)
    test_responses = pd.DataFrame(test_predictions, columns=['at_least_one', 'at_least_two', 'at_least_three'])
    test_answers = y_test.reset_index(drop=True)

    train_error = get_smoothed_mean_log_accuracy_ratio(train_answers, train_responses)
    test_error = get_smoothed_mean_log_accuracy_ratio(test_answers, test_responses)

    error_res[model_name] = {
        'best_params': grid_search.best_params_,
        'train_error': train_error,
        'test_error': test_error
    }

for model_name, errors in error_res.items():
    print(f"{model_name}: Best Params = {errors['best_params']}, Train Error = {errors['train_error']}%, Test Error = {errors['test_error']}%")

Decision Tree: Best Params = {'max_depth': 5}, Train Error = 57.94%, Test Error = 75.79%
Random Forest: Best Params = {'max_depth': 10, 'n_estimators': 50}, Train Error = 28.81%, Test Error = 55.52%


In [90]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression

In [110]:
models = {
    'K-Nearest Neighbors': (KNeighborsRegressor(), {'n_neighbors': [3, 5, 7, 10, 12, 15, 20, 50]})
}

error_res = {}

for model_name, (model, params) in models.items():
    grid_search = GridSearchCV(model, params, scoring=custom_scorer_func, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    best_model = grid_search.best_estimator_

    train_predictions = best_model.predict(X_train_scaled)
    train_responses = pd.DataFrame(train_predictions, columns=['at_least_one', 'at_least_two', 'at_least_three'])
    train_answers = y_train.reset_index(drop=True)

    test_predictions = best_model.predict(X_test_scaled)
    test_responses = pd.DataFrame(test_predictions, columns=['at_least_one', 'at_least_two', 'at_least_three'])
    test_answers = y_test.reset_index(drop=True)

    train_error = get_smoothed_mean_log_accuracy_ratio(train_answers, train_responses)
    test_error = get_smoothed_mean_log_accuracy_ratio(test_answers, test_responses)

    error_res[model_name] = {
        'best_params': grid_search.best_params_,
        'train_error': train_error,
        'test_error': test_error
    }

for model_name, errors in error_res.items():
    print(f"{model_name}: Best Params = {errors['best_params']}, Train Error = {errors['train_error']}%, Test Error = {errors['test_error']}%")

K-Nearest Neighbors: Best Params = {'n_neighbors': 3}, Train Error = 91.82%, Test Error = 159.54%


In [175]:
model = GradientBoostingRegressor(n_estimators=150, learning_rate=0.08, max_depth=7)

y_pred_train = pd.DataFrame()
y_pred_test = pd.DataFrame()

for column in y.columns:
    model.fit(X_train, y_train[column])
    y_pred_train[column] = model.predict(X_train)
    y_pred_test[column] = model.predict(X_test)

responses_train = pd.DataFrame(y_pred_train, columns=['at_least_one', 'at_least_two', 'at_least_three'])
answers_train = y_train.reset_index(drop=True)
responses_test = pd.DataFrame(y_pred_test, columns=['at_least_one', 'at_least_two', 'at_least_three'])
answers_test = y_test.reset_index(drop=True)

percentage_error_train = get_smoothed_mean_log_accuracy_ratio(answers_train, responses_train)
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(answers_test, responses_test)
print(f'Percentage Error on Train: {percentage_error_train}%')
print(f'Percentage Error on Test: {percentage_error_test}%')


Percentage Error on Train: 1.61%
Percentage Error on Test: 53.1%
